# Exploring Score DataFrame Sizes

As we combine the output DataFrames from multiple scoring methods, it is useful to know the sizes of each dataframe and how much overlap they have with one another.

In [ ]:
import os
import pandas as pd
import numpy as np
from grn_inference.normalization import minmax_normalize_pandas

project_dir = "/gpfs/Labs/Uzun/SCRIPTS/PROJECTS/2024.SINGLE_CELL_GRN_INFERENCE.MOELLER"
input_dir = os.path.join(project_dir, "input/DS011_mESC/DS011_mESC_sample1")
output_dir = os.path.join(project_dir, "output/DS011_mESC/DS011_mESC_sample1")

## Size of the Input Dataset

In [ ]:
atac_processed_df = pd.read_parquet(
    os.path.join(input_dir, "DS011_mESC_ATAC_processed.parquet"), 
    engine="pyarrow"
    )

rna_processed_df = pd.read_parquet(
    os.path.join(input_dir, "DS011_mESC_RNA_processed.parquet"), 
    engine="pyarrow"
    )

print("scATAC-seq Dataset:")
print(f"  - Cells: {atac_processed_df.shape[1]-1}")
print(f"  - Peaks: {atac_processed_df.shape[0]}")

print("\nscRNA-seq Dataset:")
print(f"  - Cells: {rna_processed_df.shape[1]-1}")
print(f"  - Genes: {rna_processed_df.shape[0]}")

---

## Peak to TG

The peak to TG regulatory potential methods we use are:

1) Cicero
2) TSS Distance
3) MIRA
4) TG Expression / Peak Accessibility correlation

First, let's look at the size of each methods output

In [ ]:
def peak_to_tg_score_summary(score_file_name, method_name) -> pd.DataFrame:
    df = pd.read_parquet(
        os.path.join(output_dir, score_file_name)
    )
    
    print(f"{method_name}:")
    print(f"  - Edges: {df.shape[0]:,}")
    print(f"  - Unique Peaks: {df['peak_id'].nunique():,}")
    print(f"  - Unique TGs: {df['target_id'].nunique():,}\n")
    
    return df
    
def tf_to_peak_score_summary(score_file_name, method_name) -> pd.DataFrame:
    df = pd.read_parquet(
        os.path.join(output_dir, score_file_name)
    )
    
    print(f"{method_name}:")
    print(f"  - Edges: {df.shape[0]:,}")
    print(f"  - Unique TFs: {df['source_id'].nunique():,}")
    print(f"  - Unique Peaks: {df['peak_id'].nunique():,}\n")
    
    return df

method_files = {
    "Cicero": "cicero_peak_to_tg_scores.parquet",
    "TSS Distance Score": "tss_distance_score.parquet",
    "MIRA": "ds011_peak_to_gene_lite_rp_score_chrom_diff.parquet",
    "Correlation": "peak_to_gene_correlation.parquet"
}

method_dfs = {
    name: peak_to_tg_score_summary(path, name)
    for name, path in method_files.items()
}

method_peaks = {name: set(df["peak_id"]) for name, df in method_dfs.items()}
method_targets = {name: set(df["target_id"]) for name, df in method_dfs.items()}

The TSS distance score DataFrame is much larger than the others. Let's look at how many of the peak to TG edges are shared between scoring methods.

In [ ]:
def get_shared_elements(method_dict, label):
    shared_elements = set()
    method_names = list(method_dict.keys())
    print(f"\nShared {label}s:")
    for i, m1 in enumerate(method_names):
        for m2 in method_names[i+1:]:
            shared = method_dict[m1] & method_dict[m2]
            print(f"  - {m1} vs {m2}: {len(shared):,}")
            shared_elements.update(shared)
    return shared_elements

# Compute shared peaks and target genes
shared_peaks = get_shared_elements(method_peaks, "Peak")
shared_targets = get_shared_elements(method_targets, "Target Gene")

print(f"\nPeaks in more than one method: {len(shared_peaks):,}")
print(f"Target genes in more than one method: {len(shared_targets):,}\n")


There is not a ton of overlap, but this reduces the size of the DataFrame. Now, lets subset each score DataFrame to only use edges that are in more than one scoring method.

In [ ]:
def subset_df_to_peaks_and_targets_in_more_than_one_df(df, method_name, shared_peaks, shared_targets) -> pd.DataFrame:
    df = df[
    (df["peak_id"].isin(shared_peaks)) &
    (df["target_id"].isin(shared_targets))
    ].dropna()
    
    print(f"{method_name}:")
    print(f"  - Edges: {df.shape[0]:,}")
    print(f"  - Unique Peaks: {df['peak_id'].nunique():,}")
    print(f"  - Unique TGs: {df['target_id'].nunique():,}\n")
    
    return df

# Filter each dataframe to only those peak-target pairs that are in multiple methods
method_dfs_subset = {
    name: subset_df_to_peaks_and_targets_in_more_than_one_df(df, name, shared_peaks, shared_targets)
    for name, df in method_dfs.items()
}

Once we have only edges found in more than one score DataFrame, we can merge the scoring methods together into a single tf to peak DataFrame

In [ ]:
merge_scores_no_tss = pd.merge(
    pd.merge(method_dfs_subset["Cicero"], method_dfs_subset["MIRA"], on=["peak_id", "target_id"], how="outer"),
    method_dfs_subset["Correlation"], on=["peak_id", "target_id"], how="outer"
)

tf_to_peak = pd.merge(
    merge_scores_no_tss, 
    method_dfs_subset["TSS Distance Score"], 
    on=["peak_id", "target_id"], 
    how="inner"
    )
tf_to_peak

Let's look at how sparse the dataset is

In [ ]:
total_edges = len(tf_to_peak[["peak_id", "target_id"]])
score_cols = {
    "Cicero Score": "cicero_score",
    "TSS Distance Score": "TSS_dist_score",
    "MIRA LITE Score": "MIRA_LITE_RP_score",
    "MIRA Chromatin Differential Score": "chromatin_differential",
    "Correlation Score": "correlation"
}

print(f"Total Edges: {total_edges:,}")
for label, col in score_cols.items():
    score_count = tf_to_peak[col].notna().sum()
    print(f"  - {label}: {score_count:,} ({score_count / total_edges:.2%} of total scores)")


The merged dataset isn't too sparse if we outer merge the method score DataFrames, then use an inner merge for the TSS distance scores. The number of peak to TG edges has dropped dramatically, which may or may not be a good thing. We can add a column called `"support_count"` which counts the number methods with non-nan values across rows.

In [ ]:
# Count how many scores are non-null for each edge
score_support_counts = tf_to_peak[score_cols.values()].notna().sum(axis=1)

# Select edges supported by 2 or more methods
for i, _ in enumerate(score_cols.values()):
    if i > 0:
        multi_supported_edges = tf_to_peak[score_support_counts >= i]

        print(f"Edges supported by ≥{i} methods: {len(multi_supported_edges):,} out of {len(tf_to_peak):,}")
tf_to_peak["support_count"] = score_support_counts

Now that we have a DataFrame containing all of the peak to TG scores for at least 2 methods, we can add in the **minmax normalized mean peak accessibility and mean TG expression** for each edge. We use an inner join here to only keep rows that have genes and peaks in the expression and accesibility data.

In [ ]:
mean_peak_expr_norm = minmax_normalize_pandas(
    atac_processed_df.set_index("peak_id").mean(axis=1).rename("mean_peak_expr").to_frame(),
    ["mean_peak_expr"]
).reset_index()

mean_gene_expr_norm = minmax_normalize_pandas(
    rna_processed_df.set_index("gene_id").mean(axis=1).rename("mean_gene_expr").to_frame(),
    ["mean_gene_expr"]
).reset_index()

tf_to_peak_mean_acc = pd.merge(tf_to_peak, mean_peak_expr_norm, on="peak_id", how="inner")
full_tf_to_peak = pd.merge(tf_to_peak_mean_acc, mean_gene_expr_norm, left_on="target_id", right_on="gene_id", how="inner").drop(columns="gene_id")
full_tf_to_peak

---

## TF to Peak

Now that we have the peak to TG scores together, we need to combine the TF to peak binding potential scores. We have two methods that calculate TF to peak scores:

1) Homer
2) Sliding Window

Let's start by loading these score DataFrames in and take a look at their sizes.